In [1]:
%pylab --no-import-all inline

Populating the interactive namespace from numpy and matplotlib


# Most general form of cross-validation, with history
---

This provides little personalization, and still avoids the issue of using a subject's future data for prediction.

In [2]:
from os import path
import sys

import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_predict

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = path.join("..", 'src')
sys.path.append(src_dir)

# import my method from the source code
%aimport features.build_features
%aimport models.fit_predict
%aimport visualization.visualize
from features.build_features import previous_value
from models.fit_predict import cv_predict
from visualization.visualize import modified_bland_altman_plot, residual_plot

In [3]:
file = path.join("..", "data", "interim", "df.csv")
df = pd.read_csv(file, index_col=0)

## Compute features

In [4]:
features = ["AGE"]

### Previous `L_PREOVULATION` and `L_CYCLE`

In [5]:
df['past_L_PREOVULATION'] = previous_value('L_PREOVULATION', df)
df['past_L_CYCLE'] = previous_value('L_CYCLE', df)

df.dropna(subset=[
    'past_L_PREOVULATION', 
    'past_L_CYCLE'
], inplace=True)

features += ['past_L_PREOVULATION', 'past_L_CYCLE']

### $n$ days of temperature measurements.

The use case requires deleting those whose ovulation occurs before these $n$ days.

In [6]:
NUMBER_OF_DAYS = 10
df = df[df.L_PREOVULATION > NUMBER_OF_DAYS]  # No use predicting backward in time.
temp_measurements = ["TEMP" + str(i + 1) for i in range(NUMBER_OF_DAYS)]
features += temp_measurements

In [7]:
features

['AGE',
 'past_L_PREOVULATION',
 'past_L_CYCLE',
 'TEMP1',
 'TEMP2',
 'TEMP3',
 'TEMP4',
 'TEMP5',
 'TEMP6',
 'TEMP7',
 'TEMP8',
 'TEMP9',
 'TEMP10']

In [8]:
X = df[features]
y = df.L_PREOVULATION
grouping = df.ID

## Perform regression

In [9]:
y_pred = cv_predict(X, y, groups=grouping)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.2s finished


In [11]:
mean_squared_error(y_pred=y_pred, y_true=y)

14.863430587190328

In [12]:
mean_absolute_error(y_pred=y_pred, y_true=y)

2.7331466205322847

In [ ]:
modified_bland_altman_plot(y_pred, y);

In [ ]:
residual_plot(y_pred, y);

In [ ]:
y_pred

## Discussion
---

Our features are only the first ten temperatures of the cycle and the participant's last cycle length and follicular phase length. With it, we achieve a MSE of about 12, which beats the Bortot paper's 15. In terms of use case, this is about equal to the Bortot result.

In [ ]:
df.L_PERIOD.median()

Now, the median period length is 5, which means that we are really using measurements of BBT during the period to determine the day of ovulation.

Since this model has only slight personalization, it's exciting to see how well a personalized model will do.